In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from os.path import join as oj
import analyze_utils
import matplotlib.pyplot as plt
import iprompt
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
tqdm.pandas()

In [29]:
keys = ['model_cls', 'task_name', 'checkpoint', 'seed']
keys_extra = ['n_shots', 'num_learned_tokens']
keys_out = ['reciprocal_rank', 'prefixes', 'iprompt_preprefix_str']

save_dir = '/home/chansingh/mntv1/iprompt_revision_xmas/'

# r = analyze_utils.load_results_and_cache_autoprompt_json(save_dir, save_file='r.pkl')
r = pd.read_pickle(oj(save_dir, 'r.pkl'))

In [30]:
# # subselect the data
# r = r[r['n_shots'] == 5]
# r = r[r['num_learned_tokens'] == 6]

# # add more keys
# keys_derived = ['task_collection', 'top_prompt_correctness', 'bleu_top_prompt']
# r['task_collection'] = r.task_name.apply(analyze_utils.task_collection)
# r['top_prompt_correctness'] = r['reciprocal_rank'] == 1
# r['gt_prompt'] = r['task_name'].apply(lambda x: iprompt.data.TASKS[x]['description'])
# r['bleu_top_prompt'] = r.progress_apply(lambda x: sentence_bleu(x['gt_prompt'].split(), x['prefixes'].split()), axis=1) # maybe need to split before calling?

# d = r[keys + keys_extra + keys_out + keys_derived]
# d.to_pickle(oj(save_dir, 'd.pkl'))
d  = pd.read_pickle(oj(save_dir, 'd.pkl'))

(419705, 60)

In [31]:
d.shape

(419705, 12)

# Metrics table

In [10]:
metrics = d.groupby(by=['model_cls', 'task_collection', 'checkpoint']).mean(numeric_only=True).reset_index()
# with pd.option_context('display.float_format', lambda x: '%.1e' % x):
with pd.option_context('display.float_format', lambda x: '%.2g' % x):
    for ckpt in metrics.checkpoint.unique():
        print(ckpt)
        m = (
            metrics[metrics.checkpoint == ckpt]
            .pivot_table(index='model_cls', columns='task_collection', values=['reciprocal_rank', 'top_prompt_correctness', 'bleu_top_prompt'])
            .transpose()
        )
        display(m)

EleutherAI/gpt-j-6B


model_cls                               autoprompt  iprompt   suffix
                       task_collection                              
bleu_top_prompt        ANLI               5.8e-234 2.4e-232 1.1e-232
                       DD                 3.5e-235 8.1e-233 5.6e-233
                       Induction          2.8e-233 5.4e-159 7.5e-233
                       Math               2.8e-234 3.1e-232   3e-232
reciprocal_rank        ANLI                  0.054     0.43    1e-10
                       DD                  0.00034    0.009     0.05
                       Induction             0.099     0.35    0.071
                       Math                   0.12      0.7    0.052
top_prompt_correctness ANLI                  0.054     0.39        0
                       DD                        0        0    0.017
                       Induction             0.092     0.28    0.071
                       Math                   0.12     0.61    0.042

facebook/galactica-6.7b


model_cls                               autoprompt  iprompt   suffix
                       task_collection                              
bleu_top_prompt        ANLI               4.7e-234 1.8e-232 2.9e-232
                       DD                 4.5e-234 2.1e-232 1.6e-232
                       Induction          3.2e-233   7e-233 1.2e-232
                       Math                 8e-235 1.4e-232        0
reciprocal_rank        ANLI                 0.0051     0.58    1e-10
                       DD                    1e-10     0.26    0.026
                       Induction              0.12      0.2    1e-10
                       Math                   0.07     0.47     0.33
top_prompt_correctness ANLI                      0     0.52        0
                       DD                        0     0.16        0
                       Induction              0.11    0.062        0
                       Math                  0.042     0.39     0.33

google/flan-t5-xl


model_cls                               iprompt   suffix
                       task_collection                  
bleu_top_prompt        ANLI            1.5e-158 1.2e-232
                       DD              6.4e-233 5.9e-233
                       Induction        3.8e-81 1.2e-232
                       Math            3.2e-159 1.1e-232
reciprocal_rank        ANLI                0.27    1e-10
                       DD                 0.074    0.034
                       Induction           0.16    0.014
                       Math                0.16    1e-10
top_prompt_correctness ANLI                0.21        0
                       DD                 0.059    0.023
                       Induction          0.099        0
                       Math                 0.1        0

# Prompts table

In [34]:
for ckpt in d.checkpoint.unique():
    print(ckpt)
    prompts = d[d.checkpoint == ckpt]
    print(prompts.shape[0])
    prompts = (
        prompts
        .sort_values(by=['task_collection', 'reciprocal_rank'], ascending=False)
        .groupby(by=['model_cls', 'task_name'])
        .first()
    )
    # with pd.option_context('display.float_format', lambda x: '%.1e' % x):
    with pd.option_context('display.max_rows', None):
        display(prompts[['prefixes']].head())

EleutherAI/gpt-j-6B
246204


prefixes
model_cls  task_name                                                       
autoprompt active_to_passive                            (= 18 the the subst
           add_two                 >:Returns Adding togetherFont accomplish
           antonyms                                      čthe bectheBut But
           cause_and_effect      REG Kinect virginity developed mosquit The
           common_concept     ???????? parted configuredthe        ????????

google/flan-t5-xl
134666


prefixes
model_cls task_name                                          
iprompt   active_to_passive  Answer: The judge encouraged the
          add_two                     prompt: find the sum of
          antonyms               What is the opposite of what
          cause_and_effect                    Answer: Mary's.
          common_concept              answer: sewing wrenches

facebook/galactica-6.7b
38835


prefixes
model_cls  task_name                                                  
autoprompt add_two                                ě addthe Hyper� addi
           antonyms           meet equilibration stiptertead asymmetry
           cause_and_effect                      shaking Dthethethethe
           d3_26                                    NeuthePreftheDEthe
           d3_4                  MULT semilinear unarybuffer Gior fate

In [28]:
d.head(n=200)

,model_cls,task_name,checkpoint,seed,n_shots,num_learned_tokens,reciprocal_rank,prefixes,iprompt_preprefix_str,task_collection,top_prompt_correctness,bleu_top_prompt
0,iprompt,task1146_country_capital,EleutherAI/gpt-j-6B,1,5,6,1.0,Which city is the capital and,NaN,ANLI,True,0.000000e+00
1,iprompt,task1146_country_capital,EleutherAI/gpt-j-6B,1,5,6,1.0,Which city is the capital of,NaN,ANLI,True,0.000000e+00
2,iprompt,task1146_country_capital,EleutherAI/gpt-j-6B,1,5,6,1.0,Which country is the capital of,NaN,ANLI,True,0.000000e+00
3,iprompt,task1146_country_capital,EleutherAI/gpt-j-6B,1,5,6,1.0,Which country's capital is located,NaN,ANLI,True,0.000000e+00
4,iprompt,task1146_country_capital,EleutherAI/gpt-j-6B,1,5,6,1.0,Which country's capital city is,NaN,ANLI,True,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...
195,iprompt,task1146_country_capital,EleutherAI/gpt-j-6B,1,5,6,1.0,Enter a single city from your,NaN,ANLI,True,1.164047e-231
196,iprompt,task1146_country_capital,EleutherAI/gpt-j-6B,1,5,6,1.0,Type out your name and the,NaN,ANLI,True,0.000000e+00
197,iprompt,task1146_country_capital,EleutherAI/gpt-j-6B,1,5,6,1.0,Please choose the city of your,NaN,ANLI,True,0.000000e+00
198,iprompt,task1146_country_capital,EleutherAI/gpt-j-6B,1,5,6,1.0,Which city in the country is,NaN,ANLI,True,0.000000e+00


In [19]:
for k, g in prompts:
    print(k)
    display(g.head())

('autoprompt', 'add_two')


,model_cls,task_name,checkpoint,seed,n_shots,num_learned_tokens,reciprocal_rank,prefixes,iprompt_preprefix_str,task_collection,top_prompt_correctness,bleu_top_prompt
0,autoprompt,add_two,facebook/galactica-6.7b,1,5,6,1.0,ě addthe Hyper� addi,,Math,True,0.0
1,autoprompt,add_two,facebook/galactica-6.7b,1,5,6,1.0,ě addthe Rep� addi,,Math,True,0.0
2,autoprompt,add_two,facebook/galactica-6.7b,1,5,6,1.0,ě adding restrrbracket$ addi,,Math,True,0.0
3,autoprompt,add_two,facebook/galactica-6.7b,1,5,6,1.0,ě adding rest countable� addi,,Math,True,0.0
4,autoprompt,add_two,facebook/galactica-6.7b,1,5,6,1.0,ě adding rest κ� addi,,Math,True,0.0


('autoprompt', 'antonyms')


,model_cls,task_name,checkpoint,seed,n_shots,num_learned_tokens,reciprocal_rank,prefixes,iprompt_preprefix_str,task_collection,top_prompt_correctness,bleu_top_prompt
0,autoprompt,antonyms,facebook/galactica-6.7b,3,5,6,1.000000e-10,meet equilibration stiptertead asymmetry,,Induction,False,0.0
1,autoprompt,antonyms,facebook/galactica-6.7b,3,5,6,1.000000e-10,meet reinforcement stip Paired continu asymmetry,,Induction,False,0.0
2,autoprompt,antonyms,facebook/galactica-6.7b,3,5,6,1.000000e-10,meettert stipsubjectsead asymmetry,,Induction,False,0.0
3,autoprompt,antonyms,facebook/galactica-6.7b,3,5,6,1.000000e-10,consequences Rayleigh conven Humans reversethe,,Induction,False,0.0
4,autoprompt,antonyms,facebook/galactica-6.7b,3,5,6,1.000000e-10,ovesorem interpre Humansenerate asymmetry,,Induction,False,0.0


('autoprompt', 'cause_and_effect')


...

,model_cls,task_name,checkpoint,seed,n_shots,num_learned_tokens,reciprocal_rank,prefixes,iprompt_preprefix_str,task_collection,top_prompt_correctness,bleu_top_prompt
0,suffix,taxonomy_animal,facebook/galactica-6.7b,3,5,6,1.000000e-10,[Image](..,,Induction,False,0.000000e+00
1,suffix,taxonomy_animal,facebook/galactica-6.7b,3,5,6,1.000000e-10,Weren't there more,,Induction,False,0.000000e+00
2,suffix,taxonomy_animal,facebook/galactica-6.7b,3,5,6,1.000000e-10,{} is a,,Induction,False,1.384293e-231
3,suffix,taxonomy_animal,facebook/galactica-6.7b,3,5,6,1.000000e-10,what color are you wearing?,,Induction,False,0.000000e+00
0,suffix,taxonomy_animal,facebook/galactica-6.7b,2,5,6,1.000000e-10,The following are examples of human,,Induction,False,0.000000e+00


('suffix', 'translation_en-es')


,model_cls,task_name,checkpoint,seed,n_shots,num_learned_tokens,reciprocal_rank,prefixes,iprompt_preprefix_str,task_collection,top_prompt_correctness,bleu_top_prompt
0,suffix,translation_en-es,facebook/galactica-6.7b,3,5,6,1.000000e-10,Quizlet test with code,,Induction,False,0.000000e+00
1,suffix,translation_en-es,facebook/galactica-6.7b,3,5,6,1.000000e-10,Quiz 2 Input Questions,,Induction,False,0.000000e+00
2,suffix,translation_en-es,facebook/galactica-6.7b,3,5,6,1.000000e-10,A man is trying to sell,,Induction,False,0.000000e+00
3,suffix,translation_en-es,facebook/galactica-6.7b,3,5,6,1.000000e-10,A man asks a robot to,,Induction,False,9.853445e-232
0,suffix,translation_en-es,facebook/galactica-6.7b,1,5,6,1.000000e-10,(answer option is presented,,Induction,False,0.000000e+00


('suffix', 'word_in_context')


,model_cls,task_name,checkpoint,seed,n_shots,num_learned_tokens,reciprocal_rank,prefixes,iprompt_preprefix_str,task_collection,top_prompt_correctness,bleu_top_prompt
0,suffix,word_in_context,facebook/galactica-6.7b,1,5,6,1.000000e-10,"Given two sentences, decide if",,Induction,False,0.0
1,suffix,word_in_context,facebook/galactica-6.7b,1,5,6,1.000000e-10,"Given two sentences, identify similar",,Induction,False,0.0
2,suffix,word_in_context,facebook/galactica-6.7b,1,5,6,1.000000e-10,Given an input sentence pair and,,Induction,False,0.0
3,suffix,word_in_context,facebook/galactica-6.7b,1,5,6,1.000000e-10,"Given an input sentence pair,",,Induction,False,0.0


In [13]:
d.columns

Index(['model_cls', 'task_name', 'checkpoint', 'seed', 'n_shots',
       'num_learned_tokens', 'reciprocal_rank', 'prefixes',
       'iprompt_preprefix_str', 'task_collection', 'top_prompt_correctness',
       'bleu_top_prompt'],
      dtype='object')

In [ ]:
group_keys = ['model_cls', 'task_name', 'num_learned_tokens', 'n_shots', 'checkpoint', 'seed']
# metric = 'prefix_test_acc'
df = d[d['model_cls'] == 'iprompt']
with pd.option_context('display.max_rows', None):
    # display(r.groupby(group_keys)['prefixes'].max())
    display(df.groupby(group_keys).first()[['prefixes']].sort_values(by=['model_cls', 'task_name']).iloc[::-1])

In [33]:
df.groupby(group_keys).size()

model_cls  task_name          num_learned_tokens  n_shots  checkpoint               seed
iprompt    active_to_passive  6                   5        EleutherAI/gpt-j-6B      1        313
                                                                                    2        538
                                                                                    3        616
                                                           facebook/galactica-6.7b  3        543
                                                           google/flan-t5-xl        1       1024
                                                                                            ... 
           word_in_context    6                   5        EleutherAI/gpt-j-6B      2        506
                                                                                    3        520
                                                           facebook/galactica-6.7b  3        290
                                      